# Sistem Rekomendasi Film Menggunakan Content-based Filtering

Notebook ini menjelaskan langkah-langkah untuk membangun sistem rekomendasi film menggunakan pendekatan **Content-based Filtering**. Sistem ini akan merekomendasikan film kepada pengguna berdasarkan deskripsi dan genre film yang telah mereka tonton sebelumnya.

## Sesi 1: Importing Libraries


# **Importing Libraries**
### Penjelasan:
- **pandas**: Pustaka ini digunakan untuk manipulasi dan analisis data. Kita akan menggunakannya untuk memuat dataset dan melakukan operasi pada DataFrame.
- **TfidfVectorizer**: Digunakan untuk mengubah teks menjadi representasi numerik menggunakan metode TF-IDF (Term Frequency-Inverse Document Frequency), yang membantu kita memahami pentingnya kata-kata dalam dokumen.
- **cosine_similarity**: Fungsi ini digunakan untuk menghitung kemiripan antar vektor, yang dalam konteks ini adalah kemiripan antar film berdasarkan deskripsi mereka.




In [1]:
# Importing libraries
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# **Data Collection**
### Penjelasan:
- Di sini, kita memuat dataset film dari file CSV bernama `movies.csv`. Dataset ini harus berisi kolom yang relevan seperti `title` dan `description`. Pastikan file tersebut berada di direktori yang sama dengan notebook ini agar dapat diakses dengan benar.




In [2]:
# Step 1: Data Collection
# Load the dataset
data = pd.read_csv('/content/tmdb_5000_movies.csv')  # Pastikan file 'movies.csv' ada di direktori yang sama

# **Data Understanding**
### Penjelasan:
- **data.head()**: Menampilkan lima baris pertama dari dataset untuk memberikan gambaran umum tentang data yang kita miliki.
- **data.info()**: Memberikan informasi tentang kolom dalam dataset, termasuk jumlah entri, tipe data, dan jumlah nilai non-null. Ini membantu kita memahami struktur dan kualitas data.




In [3]:
# Step 2: Data Understanding
print("Dataset Overview:")
print(data.head())
print("\nDataset Info:")
print(data.info())

Dataset Overview:
      budget                                             genres  \
0  237000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
1  300000000  [{"id": 12, "name": "Adventure"}, {"id": 14, "...   
2  245000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
3  250000000  [{"id": 28, "name": "Action"}, {"id": 80, "nam...   
4  260000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   

                                       homepage      id  \
0                   http://www.avatarmovie.com/   19995   
1  http://disney.go.com/disneypictures/pirates/     285   
2   http://www.sonypictures.com/movies/spectre/  206647   
3            http://www.thedarkknightrises.com/   49026   
4          http://movies.disney.com/john-carter   49529   

                                            keywords original_language  \
0  [{"id": 1463, "name": "culture clash"}, {"id":...                en   
1  [{"id": 270, "name": "ocean"}, {"id": 726, "na...                en   
2 

# **Data Preparation**
### Penjelasan:
- **dropna()**: Menghapus baris yang memiliki nilai kosong pada kolom `description` dan `genres`. Ini penting untuk memastikan bahwa kita hanya bekerja dengan data yang lengkap, karena nilai kosong dapat menyebabkan kesalahan saat melakukan analisis.
- **data.shape**: Menampilkan dimensi dataset setelah pembersihan, memberikan informasi tentang jumlah baris dan kolom yang tersisa.



In [4]:
# Step 3: Data Preparation
# Cleaning the data by removing rows with missing values in 'overview' and 'genres'
data.dropna(subset=['overview', 'genres'], inplace=True)

# Display the shape of the dataset after cleaning
print("\nShape of dataset after cleaning:", data.shape)


Shape of dataset after cleaning: (4800, 20)


# **Vectorization using TF-IDF**
### Penjelasan:
- **TfidfVectorizer(stop_words='english')**: Membuat objek TF-IDF vectorizer dengan mengabaikan kata-kata umum dalam bahasa Inggris (stop words) yang tidak memberikan informasi penting.
- **fit_transform()**: Menerapkan TF-IDF pada kolom `description` dari dataset, menghasilkan matriks TF-IDF yang merepresentasikan deskripsi film dalam bentuk numerik. Setiap baris mewakili film, dan setiap kolom mewakili kata unik dalam deskripsi.



In [5]:
# Step 4: Vectorization using TF-IDF
tfidf = TfidfVectorizer(stop_words='english')
# Change 'description' to 'overview' as it contains the movie descriptions
tfidf_matrix = tfidf.fit_transform(data['overview'])

# **Computing Cosine Similarity**
### Penjelasan:
- **cosine_similarity()**: Menghitung kemiripan kosinus antara semua vektor dalam matriks TF-IDF. Hasilnya adalah matriks di mana setiap elemen (i, j) menunjukkan seberapa mirip film i dengan film j. Nilai berkisar antara -1 (sangat tidak mirip) hingga 1 (sangat mirip).




In [6]:
# Step 5: Computing Cosine Similarity
cosine_sim = cosine_similarity(tfidf_matrix)

# **Creating the Recommendation Function**
### Penjelasan:
- Fungsi `recommend_movies(title)` menerima judul film sebagai input.
- **idx**: Mencari indeks film berdasarkan judul yang diberikan.
- **sim_scores**: Mengambil skor kemiripan untuk semua film dibandingkan dengan film input.
- **sorted()**: Mengurutkan skor kemiripan dari yang tertinggi ke terendah.
- **sim_scores[1:6]**: Mengambil lima rekomendasi teratas, mengecualikan film itu sendiri.
- **movie_indices**: Mengambil indeks dari lima film teratas.
- Fungsi mengembalikan daftar judul film rekomendasi berdasarkan kemiripan.




In [7]:
# Step 6: Creating the Recommendation Function
def recommend_movies(title, cosine_sim=cosine_sim):
    # Get the index of the movie that matches the title
    idx = data.index[data['title'] == title][0]

    # Get the pairwise similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 5 most similar movies (excluding itself)
    sim_scores = sim_scores[1:6]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    return data['title'].iloc[movie_indices]

# **Testing the Recommendation System**
### Penjelasan:
- Bagian ini menjalankan fungsi rekomendasi dengan judul film contoh ("The Matrix").
- Anda dapat mengganti variabel `test_movie` dengan judul lain untuk mendapatkan rekomendasi berbeda.
- Hasilnya akan menampilkan daftar lima film yang direkomendasikan berdasarkan kesamaan dengan film input.

---



In [8]:
# Step 7: Testing the Recommendation System
if __name__ == "__main__":
    test_movie = "Avatar"  # Ganti dengan judul film yang ingin direkomendasikan
    print(f"\nRekomendasi film untuk '{test_movie}':")
    recommended_movies = recommend_movies(test_movie)
    print(recommended_movies)


Rekomendasi film untuk 'Avatar':
3604               Apollo 18
2130            The American
634               The Matrix
1341    The Inhabited Island
529         Tears of the Sun
Name: title, dtype: object


In [10]:
# Evaluation - Metrik Precision
def calculate_precision(recommended, relevant):

    true_positives = len(set(recommended) & set(relevant))
    precision = true_positives / len(recommended) if len(recommended) > 0 else 0
    return precision

# Contoh penggunaan untuk menghitung precision
relevant_movies = ["Apollo 18", "The American", "The Matrix", "The Inhabited Island", "Tears of the Sun"]  # Misalkan ini adalah film relevan bagi pengguna
recommended_movies_list = recommended_movies.tolist()  # Mengubah hasil rekomendasi menjadi list

precision_score = calculate_precision(recommended_movies_list, relevant_movies)
print(f"\nPrecision Score: {precision_score:.2f} (atau {precision_score * 100:.2f}%)")


Precision Score: 1.00 (atau 100.00%)
